In [243]:
# YoutubeModule Class
from CrawlingModule import YoutubeBuilder
import pandas as pd
import Preprocessing as prep
import numpy as np

# KEY 파일로 읽어오기
KEY_FILE = open('./API_KEY.txt', 'r')

# API_KEY 할당
API_KEY = KEY_FILE.readline().split(':')[1]

# YoutbeModule Instarnce 
bulider = YoutubeBuilder(API_KEY)

In [244]:
# channelId = bulider.search_get_channelId('할명수')
# video_id_list = bulider.search_get_videoId_in_channel(channelId=channelId)

# video_list = bulider.get_videos_in_videoId_list(videoId_list=video_id_list)

In [245]:
# comments_list, none_video_id_list = bulider.get_comments(video_id_list)

In [ ]:
df = pd.DataFrame(video_id_list, columns=['video_id'])
(df.video_id.value_counts() > 2).values

In [ ]:
# video_id,category_id,category_name,title,views_count,like_count,uploaded_at
df = pd.DataFrame(video_list, columns=['video_id', 'category_id', 'category_name', 'title', 'views_count', 'like_count', 'uploaded_at', 'tags'])

df.tags = chim_df.tags.apply(lambda x : prep.extract_tags(x, name='#할명수 '))
df.uploaded_at = pd.to_datetime(chim_df.uploaded_at)
df.to_csv('./데이터/할명수_videos.csv', index=False)

In [ ]:
df = pd.DataFrame(comments_list, columns=['video_id', 'comment_id', 'comment', 'like_count', 'created_at', 'updated_at'])
df.created_at = pd.to_datetime(df.created_at)
df.updated_at = pd.to_datetime(df.updated_at)


df.to_csv('./데이터/할명수_comments.csv', index=False)
# df.isna().sum()

In [263]:
# df = pd.read_csv('C:/Users/TECH2_06/Downloads/아구이뽀_videos.csv', engine='python', encoding='utf-8')
chim_comment_df = pd.read_csv('./데이터/침착맨_comments.csv', encoding='utf-8', engine='python')
# df.tags = df.tags.apply(lambda x : prep.extract_tags(x, name='#할명수 '))
# df.loc[df.title.apply(prep.extract_tags) != ''].tags = df.loc[df.title.apply(prep.extract_tags) != ''].title.apply(lambda x : prep.extract_tags(x, name='#할명수 #박명수 '))
# df.loc[df.title.apply(prep.extract_tags) != ''].tags
# df.loc[df.title.apply(prep.extract_tags) != ''].title.apply(lambda x : prep.extract_tags(x, name='#할명수 #박명수 '))
# df.tags = df.tags.apply(lambda x : prep.extract_tags(x, name='#슈카월드 '))

# df.uploaded_at = pd.to_datetime(df.uploaded_at)

# df.to_csv('./데이터/임나은_videos.csv', index=False)

# prep.extract_tags(df.iloc[0].tags, name='#짐종국 ')
# df.tags = df.tags.apply(lambda x : prep.extract_tags(x))
# df.sort_values(by='uploaded_at', ascending=False)

In [264]:
import pandas as pd
import numpy as np
import Preprocessing as prep
from konlpy.tag import Twitter
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from PIL import Image
from collections import Counter

In [265]:
video_comment_list = list()
unique_video_id = chim_comment_df.video_id.unique()

video_id_and_count = dict()

for video_id in unique_video_id:
    video_id_and_count[video_id] = chim_comment_df.loc[chim_comment_df.video_id == video_id].like_count.sum()

In [266]:
# 비디오별 댓글 좋아요 수
comments_like_sum = sorted(video_id_and_count.items(), key=lambda x:x[1], reverse=True)

video_on_comments_like_sum = list()

for idx, value in enumerate(comments_like_sum):
    if idx == 10:
        break
    video_on_comments_like_sum.append(value)

video_on_comments_like_sum

[('9LQsLPsKRq8', 249293),
 ('53XqM_ofaKw', 190531),
 ('IfOSBmBCWw0', 179334),
 ('Z1vgOoIuP5M', 178658),
 ('pkXiOnXAO90', 159794),
 ('IxaOybVE50Y', 157480),
 ('hnanNlDbsE4', 157341),
 ('WeGDixN-u48', 138470),
 ('V1ryDPaC6mo', 124858),
 ('SP-LJqVgQuw', 122089)]

In [ ]:
twitter = Twitter()
test_list = []
new_df = list()
stopwords = prep.stopwords('./데이터/stopwords.txt')
for video_id in video_on_comments_like_sum:
    print(chim_comment_df.loc[chim_comment_df.video_id == video_id[0]].comment.apply(lambda x : prep.text_cleaning(x, stopwords)))

341671                             [진짜, 괴한]
341672                                   []
341673        [배속, 누가, 생각, 죽겠네, 취한, 소리, 느낌]
341674                             [영어, 제목]
341675                     [세월, 영상, 벌써, 전이]
                        ...                
345305                   [오른쪽, 복숭아, 아인, 요물]
345306                            [썸네일, 보이]
345307                                   []
345308    [틀니, 복숭아, 사람, 이빨, 시험, 이빨, 박살, 틀니]
345309                                [복숭아]
Name: comment, Length: 3639, dtype: object
291412                                 [베프]
291413    [방장, 이의, 사춘기, 시작, 이제, 만날, 몹시, 읍니]
291414                                   []
291415                         [초반, 세트, 기네]
291416                                   []
                        ...                
292964                             [사람, 입장]
292965                             [인트, 천재]
292966                                 [이건]
292967                                   []
292968                           

In [ ]:
count = 0
text_list = list()
string = ''
font_path = r'C:/Windows/Fonts/malgun.ttf'
youtube_mask = np.array(Image.open('./데이터/youtube_logo.jfif'))
new_df = list()

for i in range(10):
    new_df = pd.read_csv(f'./데이터/test{i}.csv')

for comment in new_df.comment.values:
    string += ' '.join(comment)
text_list.append(string)

wc = WordCloud(font_path=font_path, background_color='white', stopwords=['개소리', '존나', '개'], mask=youtube_mask,
       max_font_size=30, scale=7).generate(text_list[0])
plt.figure(figsize=(10, 5))
plt.axis('off')
plt.imshow(wc, interpolation='bilinear')
plt.show()
# wc.to_file(filename="침착맨_원피스.png")

In [ ]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

from PIL import Image


font_path = r'C:/Windows/Fonts/malgun.ttf'

youtube_mask = np.array(Image.open('./데이터/youtube_logo.jfif'))

wc = WordCloud(font_path=font_path, background_color='white', stopwords=['개소리', '존나', '개'], mask=youtube_mask,
               max_font_size=30, scale=7, colors=['red', '']).generate(text_list[0])
plt.figure(figsize=(10, 5))
plt.axis('off')
plt.imshow(wc, interpolation='bilinear')
plt.show()
# wc.to_file(filename="침착맨_복숭아_대전.png")